# 0 - collection des données


Au lieu d'utiliser l'interface de stackoverflow, il est possible de récuperer directement dans un dataframe avec une requete python sur l'api, puis d'exporter pour traitement et modélisation.

In [137]:
import requests
import pandas as pd

#resoudre captcha sur site avant sinon error 500 'https://data.stackexchange.com/stackoverflow/query/new'
def run_query_stackoverflow(sql_query, id_captcha):

    url = f'https://data.stackexchange.com/query/save/1/{id_captcha}'
    data = {"title": "P05", "description": "", "sql" : sql_query, "g-recaptcha-response" : ""}

    response = requests.post(url, data)
    job_id = response.json().get('job_id')
    
    if(job_id is not None) :
        url = f'https://data.stackexchange.com/query/job/{job_id}'
        response = requests.get(url)

    data = response.json().get('resultSets')[0].get('rows')
    columns = [ col['name'] for col in response.json().get('resultSets')[0].get('columns')]
    
    df = pd.DataFrame(data , columns=columns)

    return df

In [138]:
df_posts = run_query_stackoverflow('SELECT Id,Body,Title,Tags FROM posts WHERE Id < 100000 AND ParentId IS NULL', id_captcha=1381588)

In [139]:
df_posts.sample(10)

,Id,Body,Title,Tags
9365,83260,"<p>Say I've got this array:\nMyArray(0)=""aaa""\...",How can I get the unique values of an array in...,<.net><arrays><unique>
4981,46276,<p>I am a web-developer working in PHP. I have...,Test Driven Development in PHP,<php><unit-testing><tdd>
2192,22552,<p>I have a regex that is going to end up bein...,"Passing a commented, multi-line (freespace) re...",<php><regex>
11102,99542,<p>How do I correct for floating point error i...,Point-Triangle Collision Detection in 3D,<math><3d><collision-detection>
2819,28756,<p>Whats the best/easiest way to obtain a coun...,The best way to get a count of IEnumerable<T>,<c#><linq>
1040,10727,"<p>In your applications, what's a ""long time"" ...","What is a ""reasonable"" length of time to keep ...",<sql><cursors>
5390,49503,<p>Can anyone recommend good tools for reverse...,Reverse engineer (oracle) schema to ERD,<oracle><uml><reverse-engineering>
8488,75246,<p>What are the implications of running a Micr...,What are the implications of running a Microso...,<ms-access><vba>
7182,63090,"<p>Here we go again, the old argument still ar...",Surrogate vs. natural/business keys,<database><database-design><primary-key><key>
10264,91683,<p>I believe strongly in using unit-tests as p...,How do you implement unit-testing in large sca...,<c++><unit-testing><coding-style>


In [141]:
df_posts.to_csv("data/df_posts.csv", index=False)